# 📘 Day 4 (06/11) — Pipeline Completa e Metriche Automatiche

**Obiettivo pratico (4 ore di codice)**: Integrare pre-training → SFT → RLHF in una pipeline completa, confrontare modelli, e implementare metriche automatiche (BLEU, ROUGE).

---

## 🗺️ Roadmap della lezione (240 minuti di codice)

| **Sezione** | **Contenuto** | **Tempo stimato** |
|-------------|---------------|-------------------|
| 1 | Setup e teoria pipeline RLHF | 20' |
| 2 | Integrazione pipeline completa | 40' |
| 3 | Casi d'uso RLHF reali | 30' |
| 4 | RLHF simulato su dataset preferenze | 40' |
| 5 | Confronto base vs SFT vs RLHF | 40' |
| 6 | Metriche automatiche (BLEU, ROUGE) | 50' |
| 7 | Discussione: valutazione automatica vs umana | 20' |
| **TOTALE** | | **240'** |

---

## 📚 Leggi dopo (teoria fuori orario)

### Pipeline RLHF completa
```
1. Pre-training
   - Dataset: centinaia miliardi token (web, libri, code)
   - Obiettivo: next token prediction
   - Output: modello base (es. GPT-3, LLaMA)
   ↓
2. Supervised Fine-Tuning (SFT)
   - Dataset: migliaia esempi labeled (instruction-following)
   - Obiettivo: imparare a seguire istruzioni
   - Output: modello SFT (es. GPT-3.5)
   ↓
3. Reward Model Training
   - Dataset: decine migliaia coppie preferenze
   - Obiettivo: predire preferenze umane
   - Output: reward model
   ↓
4. RL Fine-tuning (PPO/DPO)
   - Dataset: prompt + reward model
   - Obiettivo: massimizzare reward
   - Output: modello allineato (es. ChatGPT)
```

### Esempio: OpenAI ChatGPT
- **Pre-training**: GPT-3 (175B parametri, 300B token)
- **SFT**: ~13K esempi instruction-following
- **Reward Model**: ~33K coppie preferenze
- **PPO**: migliaia iterazioni
- **Costo totale stimato**: $10-20M

### Sfide pratiche
1. **Dataset costosi**: annotazione umana $1-10 per esempio
2. **Annotatori in disaccordo**: inter-annotator agreement ~70-80%
3. **Allineamento imperfetto**: modello può comunque allucinare
4. **Generalizzazione**: performance su distribuzione diversa può degradare
5. **Test continui**: necessità di monitoraggio post-deployment

### Metriche automatiche
- **BLEU**: precision n-grammi (traduzione)
- **ROUGE**: recall n-grammi (summarization)
- **Perplessità**: quanto il modello è "sorpreso" dal testo
- **Limiti**: non catturano semantica, coerenza, utilità

---

## 1️⃣ Setup e teoria pipeline RLHF (20 minuti)

In [ ]:
# Installazione librerie
!pip install torch transformers datasets nltk rouge-score matplotlib pandas numpy -q

In [ ]:
# Import librerie
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import os
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data

# Seed
SEED = 42
set_seed(SEED)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ Device: {device}")
print(f"✅ Seed: {SEED}")

In [ ]:
# Creazione directory
os.makedirs('./results_day4', exist_ok=True)
print("✅ Directory create")

---

## 2️⃣ Integrazione pipeline completa (40 minuti)

Visualizziamo la pipeline completa con esempi concreti.

In [ ]:
# Pipeline RLHF: schema visivo
pipeline_stages = {
    'Stage': ['Pre-training', 'SFT', 'Reward Model', 'RL Fine-tuning'],
    'Input': [
        'Testo raw (web, libri)',
        'Esempi instruction-following',
        'Coppie preferenze (chosen/rejected)',
        'Prompt + reward model'
    ],
    'Obiettivo': [
        'Next token prediction',
        'Seguire istruzioni',
        'Predire preferenze umane',
        'Massimizzare reward'
    ],
    'Dataset Size': [
        '100B-1T token',
        '10K-100K esempi',
        '10K-100K coppie',
        'Generato on-the-fly'
    ],
    'Costo (USD)': [
        '$1M-10M',
        '$10K-100K',
        '$50K-500K',
        '$50K-500K'
    ],
    'Tempo': [
        'Settimane-mesi',
        'Ore-giorni',
        'Giorni-settimane',
        'Giorni-settimane'
    ]
}

df_pipeline = pd.DataFrame(pipeline_stages)

print("\n📊 Pipeline RLHF Completa:\n")
print(df_pipeline.to_string(index=False))

df_pipeline.to_csv('./results_day4/pipeline_rlhf.csv', index=False)
print("\n✅ Tabella salvata in ./results_day4/pipeline_rlhf.csv")

In [ ]:
# Visualizza pipeline come grafico
fig, ax = plt.subplots(figsize=(12, 8))

stages = df_pipeline['Stage'].tolist()
y_pos = np.arange(len(stages))

# Colori per stage
colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']

# Disegna box per ogni stage
for i, (stage, color) in enumerate(zip(stages, colors)):
    ax.add_patch(plt.Rectangle((0, i), 1, 0.8, facecolor=color, alpha=0.7, edgecolor='black', linewidth=2))
    ax.text(0.5, i + 0.4, stage, ha='center', va='center', fontsize=14, fontweight='bold', color='white')
    
    # Aggiungi frecce tra stage
    if i < len(stages) - 1:
        ax.arrow(0.5, i + 0.85, 0, 0.1, head_width=0.1, head_length=0.05, fc='black', ec='black', linewidth=2)

ax.set_xlim(-0.2, 1.2)
ax.set_ylim(-0.5, len(stages))
ax.axis('off')
ax.set_title('Pipeline RLHF: Pre-training → SFT → Reward Model → RL', fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('./results_day4/pipeline_diagram.png', dpi=150, bbox_inches='tight')
print("\n✅ Diagramma pipeline salvato in ./results_day4/pipeline_diagram.png")
plt.show()

### 💡 Esempio: Pipeline OpenAI

**GPT-3 → GPT-3.5 → ChatGPT**:

1. **GPT-3** (2020): pre-training su 300B token
2. **GPT-3.5** (2022): SFT su ~13K esempi instruction-following
3. **ChatGPT** (2022): RLHF con PPO
   - Reward model trainato su ~33K preferenze
   - PPO con migliaia di iterazioni
   - Continuous improvement con user feedback

**Risultato**: modello che segue istruzioni, rifiuta richieste inappropriate, ammette errori.

---

## 3️⃣ Casi d'uso RLHF reali (30 minuti)

Analizziamo applicazioni concrete di RLHF.

In [ ]:
# Casi d'uso RLHF
use_cases = {
    'Applicazione': [
        'Chatbot Customer Care',
        'Assistenti Education',
        'Code Generation',
        'Content Moderation',
        'Medical Q&A'
    ],
    'Obiettivo RLHF': [
        'Cortesia, risoluzione problemi',
        'Chiarezza, pedagogia',
        'Correttezza, sicurezza',
        'Identificare tossicità',
        'Accuratezza, cautela'
    ],
    'Sfide': [
        'Gestire clienti arrabbiati',
        'Adattare a livello studente',
        'Evitare vulnerabilità',
        'Bias culturali',
        'Responsabilità legale'
    ],
    'Esempi': [
        'Zendesk AI, Intercom',
        'Khan Academy, Duolingo',
        'GitHub Copilot, Cursor',
        'OpenAI Moderation API',
        'Google Med-PaLM'
    ]
}

df_use_cases = pd.DataFrame(use_cases)

print("\n📊 Casi d'Uso RLHF:\n")
print(df_use_cases.to_string(index=False))

df_use_cases.to_csv('./results_day4/rlhf_use_cases.csv', index=False)
print("\n✅ Tabella salvata in ./results_day4/rlhf_use_cases.csv")

### 💡 Focus: Chatbot Customer Care

**Scenario**: e-commerce con 1M utenti/mese

**Pipeline**:
1. **Base model**: GPT-2 o LLaMA-7B
2. **SFT**: fine-tune su 10K conversazioni customer care
3. **Reward model**: train su 5K preferenze (cortesia, risoluzione)
4. **PPO/DPO**: ottimizza per massimizzare soddisfazione cliente

**Metriche di successo**:
- **CSAT** (Customer Satisfaction): >80%
- **Resolution rate**: >70%
- **Escalation rate**: <20%
- **Response time**: <5 secondi

**Costi**:
- Setup: $50K-100K
- Maintenance: $10K-20K/anno
- ROI: risparmio di $200K-500K/anno in agenti umani

---

## 4️⃣ RLHF simulato su dataset preferenze (40 minuti)

Simuliamo un training RLHF completo su dataset mock.

In [ ]:
# Carica modello base
MODEL_NAME = 'distilgpt2'

print(f"⏳ Caricamento {MODEL_NAME}...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model_base = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model_base.eval()

print(f"✅ Modello base caricato")

In [ ]:
# Simula modelli SFT e RLHF (per dimostrazione, usiamo stesso modello con noise)
# In produzione, questi sarebbero modelli trainati separatamente

model_sft = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model_sft.eval()

model_rlhf = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model_rlhf.eval()

print("✅ Modelli SFT e RLHF simulati (per dimostrazione)")
print("   Nota: in produzione, questi sarebbero modelli trainati con Day 2 e Day 3")

In [ ]:
# Confronto risposte Base vs SFT vs RLHF
print("⏳ Generazione risposte...\n")

# Prompt più espliciti
test_prompts = [
    "Question: How do I track my order? Answer:",
    "Question: What is your return policy? Answer:",
    "Question: Can I cancel my subscription? Answer:",
    "Question: The product arrived damaged, what should I do? Answer:",
    "Question: When will my refund be processed? Answer:"
]

# Funzione di generazione MOLTO robusta
def generate_clean(model, tokenizer, prompt, max_length=40):
    """Genera risposta pulita senza ripetizioni e whitespace"""
    inputs = tokenizer(prompt, return_tensors='pt')
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=40,
            repetition_penalty=2.0,  # Penalità molto forte
            no_repeat_ngram_size=2,  # Evita anche 2-gram ripetuti
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decodifica
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Estrai risposta (dopo "Answer:")
    if "Answer:" in full_text:
        parts = full_text.split("Answer:")
        if len(parts) > 1:
            response = parts[1].strip()
        else:
            response = full_text[len(prompt):].strip()
    else:
        response = full_text[len(prompt):].strip()
    
    # FILTRO AGGRESSIVO whitespace
    import re
    
    # 1. Rimuovi TUTTI i newline
    response = response.replace('\n', ' ')
    response = response.replace('\r', ' ')
    response = response.replace('\t', ' ')
    
    # 2. Rimuovi spazi multipli
    response = re.sub(r'\s+', ' ', response)
    
    # 3. Rimuovi "Question:" se presente
    response = response.replace('Question:', '')
    
    # 4. Tronca alla prima frase completa (punto, ?, !)
    sentences = re.split(r'[.!?]', response)
    if len(sentences) > 0 and sentences[0].strip():
        response = sentences[0].strip()
        # Aggiungi punto finale
        if response and not response[-1] in '.!?':
            response += '.'
    
    # 5. Limita lunghezza massima
    if len(response) > 120:
        response = response[:120].rsplit(' ', 1)[0] + '...'
    
    # 6. Fallback se vuoto
    if not response or len(response) < 5:
        response = "[Nessuna risposta generata]"
    
    return response

# Genera per ogni prompt
for prompt in test_prompts:
    # Estrai domanda
    question = prompt.replace("Question: ", "").replace(" Answer:", "")
    print(f"📝 {question}")
    print()
    
    # Base
    resp_base = generate_clean(model_base, tokenizer, prompt, max_length=30)
    print(f"   Base:  {resp_base}")
    
    # SFT
    resp_sft = generate_clean(model_sft, tokenizer, prompt, max_length=30)
    print(f"   SFT:   {resp_sft}")
    
    # RLHF
    resp_rlhf = generate_clean(model_rlhf, tokenizer, prompt, max_length=30)
    print(f"   RLHF:  {resp_rlhf}")
    
    print()
    print("-" * 80)
    print()

print("✅ Generazione completata\n")

print("💡 Nota importante:")
print("   - GPT-2 (124M parametri) non è trainato per Q&A customer service")
print("   - Le risposte possono essere generiche o non pertinenti")
print("   - In produzione: modelli 7B+ con fine-tuning specifico")
print("   - L'obiettivo è mostrare il CONCETTO della pipeline, non qualità assoluta")

In [ ]:
# Funzione per generare risposte
def generate_response(model, tokenizer, prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors='pt')
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("✅ Funzione di generazione definita")

In [ ]:
# Genera risposte con tutti i modelli
print("\n⏳ Generazione risposte...\n")

results = []

for prompt in test_prompts:
    print(f"Prompt: {prompt}\n")
    
    # Base
    base_response = generate_response(model_base, tokenizer, prompt)
    print(f"Base: {base_response}\n")
    
    # SFT
    sft_response = generate_response(model_sft, tokenizer, prompt)
    print(f"SFT: {sft_response}\n")
    
    # RLHF
    rlhf_response = generate_response(model_rlhf, tokenizer, prompt)
    print(f"RLHF: {rlhf_response}\n")
    
    print("-" * 80 + "\n")
    
    results.append({
        'prompt': prompt,
        'base': base_response,
        'sft': sft_response,
        'rlhf': rlhf_response
    })

print("✅ Generazione completata")

---

## 5️⃣ Confronto base vs SFT vs RLHF (40 minuti)

Confrontiamo i tre modelli con valutazione qualitativa.

In [ ]:
# Rubrica di valutazione qualitativa
rubric = {
    'Criterio': ['Cortesia', 'Informatività', 'Risoluzione', 'Chiarezza', 'Sicurezza'],
    'Descrizione': [
        'Tono educato e professionale',
        'Fornisce informazioni utili',
        'Risolve il problema del cliente',
        'Linguaggio chiaro e comprensibile',
        'Non fornisce info dannose/false'
    ],
    'Punteggio': ['1-5', '1-5', '1-5', '1-5', '1-5']
}

df_rubric = pd.DataFrame(rubric)

print("\n📊 Rubrica di Valutazione Qualitativa:\n")
print(df_rubric.to_string(index=False))

print("\n💡 Istruzioni:")
print("   - Valuta ogni risposta con punteggio 1-5 per ogni criterio")
print("   - 1 = pessimo, 3 = accettabile, 5 = eccellente")
print("   - Calcola media per ottenere punteggio finale")

In [ ]:
# Esempio di valutazione (simulata)
# In classe, gli studenti farebbero questa valutazione manualmente

evaluation_example = {
    'Modello': ['Base', 'SFT', 'RLHF'],
    'Cortesia': [2, 4, 5],
    'Informatività': [2, 3, 4],
    'Risoluzione': [1, 3, 4],
    'Chiarezza': [3, 4, 5],
    'Sicurezza': [3, 4, 5]
}

df_eval = pd.DataFrame(evaluation_example)
df_eval['Media'] = df_eval[['Cortesia', 'Informatività', 'Risoluzione', 'Chiarezza', 'Sicurezza']].mean(axis=1)

print("\n📊 Esempio di Valutazione (simulata):\n")
print(df_eval.to_string(index=False))

# Visualizza
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(df_eval['Modello']))
width = 0.15

criteria = ['Cortesia', 'Informatività', 'Risoluzione', 'Chiarezza', 'Sicurezza']
colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c', '#9b59b6']

for i, (criterion, color) in enumerate(zip(criteria, colors)):
    offset = width * (i - 2)
    ax.bar(x + offset, df_eval[criterion], width, label=criterion, color=color, alpha=0.7, edgecolor='black')

ax.set_xlabel('Modello')
ax.set_ylabel('Punteggio (1-5)')
ax.set_title('Confronto Qualitativo: Base vs SFT vs RLHF')
ax.set_xticks(x)
ax.set_xticklabels(df_eval['Modello'])
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 6)

plt.tight_layout()
plt.savefig('./results_day4/qualitative_comparison.png', dpi=150, bbox_inches='tight')
print("\n✅ Grafico salvato in ./results_day4/qualitative_comparison.png")
plt.show()

### 💡 Osservazioni attese:

1. **Base**: risposte generiche, a volte incoerenti, poco utili
2. **SFT**: migliora struttura e informatività, ma può mancare cortesia
3. **RLHF**: ottimizza per preferenze umane, più cortese e utile

**Nota**: in questo esempio i modelli sono simulati. Con modelli realmente trainati, le differenze sarebbero più marcate.

---

## 6️⃣ Metriche automatiche (BLEU, ROUGE) (50 minuti)

Implementiamo metriche automatiche per valutazione quantitativa.

### 📖 Teoria: BLEU

**BLEU** (Bilingual Evaluation Understudy) misura precision di n-grammi.

**Formula**:
```
BLEU = BP × exp(Σ w_n log p_n)
```
Dove:
- `p_n`: precision n-grammi (n=1,2,3,4)
- `w_n`: peso (tipicamente 1/4 per ogni n)
- `BP`: brevity penalty (penalizza risposte troppo corte)

**Range**: 0-1 (o 0-100 se moltiplicato per 100)

**Uso**: traduzione automatica, ma applicabile a generazione testo

**Limiti**:
- Non cattura semantica
- Favorisce match esatti
- Sensibile a sinonimi

In [ ]:
# Implementazione BLEU
def calculate_bleu(reference: str, candidate: str) -> float:
    """
    Calcola BLEU score.
    
    Args:
        reference: testo di riferimento
        candidate: testo generato
    
    Returns:
        BLEU score (0-1)
    """
    # Tokenizza
    reference_tokens = reference.lower().split()
    candidate_tokens = candidate.lower().split()
    
    # Smoothing per evitare 0
    smoothing = SmoothingFunction().method1
    
    # Calcola BLEU
    score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothing)
    
    return score

# Test
ref = "The cat is on the mat"
cand1 = "The cat is on the mat"  # Identico
cand2 = "The dog is on the mat"  # Diverso
cand3 = "A cat sits on a mat"    # Simile ma diverso

print("\n🧪 Test BLEU:\n")
print(f"Reference: {ref}")
print(f"Candidate 1 (identico): {cand1}")
print(f"   BLEU: {calculate_bleu(ref, cand1):.4f}\n")
print(f"Candidate 2 (1 parola diversa): {cand2}")
print(f"   BLEU: {calculate_bleu(ref, cand2):.4f}\n")
print(f"Candidate 3 (simile): {cand3}")
print(f"   BLEU: {calculate_bleu(ref, cand3):.4f}")

### 📖 Teoria: ROUGE

**ROUGE** (Recall-Oriented Understudy for Gisting Evaluation) misura recall di n-grammi.

**Varianti**:
- **ROUGE-N**: overlap n-grammi
- **ROUGE-L**: longest common subsequence
- **ROUGE-W**: weighted longest common subsequence

**Formula ROUGE-L**:
```
Recall = LCS(ref, cand) / len(ref)
Precision = LCS(ref, cand) / len(cand)
F1 = 2 × (Precision × Recall) / (Precision + Recall)
```

**Range**: 0-1

**Uso**: summarization, ma applicabile a generazione testo

**Differenza con BLEU**: ROUGE misura recall (quanto del reference è catturato), BLEU misura precision (quanto del candidate è corretto)

In [ ]:
# Implementazione ROUGE
def calculate_rouge(reference: str, candidate: str) -> dict:
    """
    Calcola ROUGE scores.
    
    Args:
        reference: testo di riferimento
        candidate: testo generato
    
    Returns:
        Dizionario con ROUGE-1, ROUGE-2, ROUGE-L
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    
    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    }

# Test
print("\n🧪 Test ROUGE:\n")
print(f"Reference: {ref}")
print(f"Candidate 1 (identico): {cand1}")
rouge1 = calculate_rouge(ref, cand1)
print(f"   ROUGE-1: {rouge1['rouge1']:.4f}, ROUGE-2: {rouge1['rouge2']:.4f}, ROUGE-L: {rouge1['rougeL']:.4f}\n")

print(f"Candidate 2 (1 parola diversa): {cand2}")
rouge2 = calculate_rouge(ref, cand2)
print(f"   ROUGE-1: {rouge2['rouge1']:.4f}, ROUGE-2: {rouge2['rouge2']:.4f}, ROUGE-L: {rouge2['rougeL']:.4f}\n")

print(f"Candidate 3 (simile): {cand3}")
rouge3 = calculate_rouge(ref, cand3)
print(f"   ROUGE-1: {rouge3['rouge1']:.4f}, ROUGE-2: {rouge3['rouge2']:.4f}, ROUGE-L: {rouge3['rougeL']:.4f}")

In [ ]:
# Applica metriche ai risultati generati
# Usiamo come reference le risposte "chosen" del dataset di preferenze

# Carica dataset preferenze
import json

with open('./data/preferences.jsonl', 'r') as f:
    preferences = [json.loads(line) for line in f]

# Calcola metriche per primi 5 esempi
metrics_results = []

for i, result in enumerate(results[:5]):
    if i < len(preferences):
        reference = preferences[i]['chosen']
        
        # BLEU
        bleu_base = calculate_bleu(reference, result['base'])
        bleu_sft = calculate_bleu(reference, result['sft'])
        bleu_rlhf = calculate_bleu(reference, result['rlhf'])
        
        # ROUGE
        rouge_base = calculate_rouge(reference, result['base'])
        rouge_sft = calculate_rouge(reference, result['sft'])
        rouge_rlhf = calculate_rouge(reference, result['rlhf'])
        
        metrics_results.append({
            'prompt': result['prompt'],
            'bleu_base': bleu_base,
            'bleu_sft': bleu_sft,
            'bleu_rlhf': bleu_rlhf,
            'rouge1_base': rouge_base['rouge1'],
            'rouge1_sft': rouge_sft['rouge1'],
            'rouge1_rlhf': rouge_rlhf['rouge1']
        })

df_metrics = pd.DataFrame(metrics_results)

print("\n📊 Metriche Automatiche:\n")
print(df_metrics[['prompt', 'bleu_base', 'bleu_sft', 'bleu_rlhf']].to_string(index=False))

# Media
print("\n📊 Media Metriche:\n")
print(f"BLEU Base: {df_metrics['bleu_base'].mean():.4f}")
print(f"BLEU SFT: {df_metrics['bleu_sft'].mean():.4f}")
print(f"BLEU RLHF: {df_metrics['bleu_rlhf'].mean():.4f}")
print(f"\nROUGE-1 Base: {df_metrics['rouge1_base'].mean():.4f}")
print(f"ROUGE-1 SFT: {df_metrics['rouge1_sft'].mean():.4f}")
print(f"ROUGE-1 RLHF: {df_metrics['rouge1_rlhf'].mean():.4f}")

In [ ]:
# Visualizza metriche
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models = ['Base', 'SFT', 'RLHF']
bleu_scores = [
    df_metrics['bleu_base'].mean(),
    df_metrics['bleu_sft'].mean(),
    df_metrics['bleu_rlhf'].mean()
]
rouge_scores = [
    df_metrics['rouge1_base'].mean(),
    df_metrics['rouge1_sft'].mean(),
    df_metrics['rouge1_rlhf'].mean()
]

# Plot 1: BLEU
axes[0].bar(models, bleu_scores, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black')
axes[0].set_ylabel('BLEU Score')
axes[0].set_title('BLEU: Base vs SFT vs RLHF')
axes[0].set_ylim(0, max(bleu_scores) * 1.2)
axes[0].grid(True, alpha=0.3, axis='y')

# Plot 2: ROUGE
axes[1].bar(models, rouge_scores, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black')
axes[1].set_ylabel('ROUGE-1 F1')
axes[1].set_title('ROUGE-1: Base vs SFT vs RLHF')
axes[1].set_ylim(0, max(rouge_scores) * 1.2)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('./results_day4/automatic_metrics.png', dpi=150, bbox_inches='tight')
print("\n✅ Grafici metriche salvati in ./results_day4/automatic_metrics.png")
plt.show()

### 💡 Interpretazione metriche:

**BLEU**:
- **>0.5**: buona qualità
- **0.3-0.5**: qualità media
- **<0.3**: qualità bassa

**ROUGE-1**:
- **>0.5**: buon overlap
- **0.3-0.5**: overlap medio
- **<0.3**: overlap basso

**Nota**: questi threshold sono indicativi e dipendono dal task.

---

## 7️⃣ Discussione: valutazione automatica vs umana (20 minuti)

### Pro metriche automatiche:
- ✅ **Velocità**: valutazione istantanea
- ✅ **Costo**: praticamente zero
- ✅ **Riproducibilità**: sempre stesso risultato
- ✅ **Scalabilità**: valuta milioni di esempi

### Contro metriche automatiche:
- ❌ **Non catturano semantica**: "Il gatto mangia il topo" vs "Il topo è mangiato dal gatto"
- ❌ **Non catturano coerenza**: testo grammaticalmente corretto ma insensato
- ❌ **Non catturano utilità**: risposta corretta ma inutile per l'utente
- ❌ **Bias verso match esatti**: penalizzano sinonimi e parafrasi

### Pro valutazione umana:
- ✅ **Cattura semantica**: comprende significato
- ✅ **Cattura coerenza**: identifica nonsense
- ✅ **Cattura utilità**: valuta se risposta aiuta davvero
- ✅ **Flessibile**: può valutare criteri complessi

### Contro valutazione umana:
- ❌ **Lenta**: ore/giorni per valutare dataset
- ❌ **Costosa**: $1-10 per valutazione
- ❌ **Soggettiva**: annotatori possono non essere d'accordo
- ❌ **Non scalabile**: impossibile valutare milioni di esempi

### Best practice: approccio combinato
1. **Metriche automatiche** per screening rapido e monitoraggio continuo
2. **Valutazione umana** su campione rappresentativo (es. 100-500 esempi)
3. **Correlazione**: verifica che metriche automatiche correlino con giudizi umani
4. **Iterazione**: usa feedback umano per migliorare reward model

### Esempi industriali:
- **Google**: usa BLEU per traduzione + valutazione umana su sample
- **OpenAI**: usa metriche automatiche + human eval su benchmark
- **Meta**: usa ROUGE per summarization + crowdsourcing per valutazione

---

## 🎯 Esercizi TODO

### Esercizio 1: Metriche aggiuntive
Implementa BERTScore (usa embeddings invece di n-grammi) e confronta con BLEU/ROUGE.

### Esercizio 2: Correlazione metriche-umani
Fai valutare 10 risposte da 3 persone, calcola BLEU/ROUGE, e verifica correlazione.

### Esercizio 3: Analisi per categoria
Raggruppa prompt per categoria (tracking, return, cancel, etc.) e confronta metriche.

### Esercizio 4: A/B test simulato
Simula A/B test tra SFT e RLHF con metriche di business (CSAT, resolution rate).

In [ ]:
# TODO: Esercizio 1 - Metriche aggiuntive
# Scrivi qui il tuo codice

In [ ]:
# TODO: Esercizio 2 - Correlazione metriche-umani
# Scrivi qui il tuo codice

In [ ]:
# TODO: Esercizio 3 - Analisi per categoria
# Scrivi qui il tuo codice

In [ ]:
# TODO: Esercizio 4 - A/B test simulato
# Scrivi qui il tuo codice

---

## 🎓 Conclusione Day 4

Oggi abbiamo:
1. ✅ Compreso la **pipeline RLHF completa**
2. ✅ Analizzato **casi d'uso reali** di RLHF
3. ✅ Simulato **RLHF su dataset preferenze**
4. ✅ Confrontato **base vs SFT vs RLHF** con valutazione qualitativa
5. ✅ Implementato **metriche automatiche** (BLEU, ROUGE)
6. ✅ Discusso **trade-off** tra valutazione automatica e umana

**Prossimi passi (Day 5)**:
- Perplessità (PPL)
- lm-eval-harness per benchmark
- SQuAD per QA evaluation
- Valutazione umana in classe

---

## 📁 File generati

```
./results_day4/
├── pipeline_rlhf.csv
├── pipeline_diagram.png
├── rlhf_use_cases.csv
├── qualitative_comparison.png
└── automatic_metrics.png
```